In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import pymongo
from sklearn import preprocessing

# SETTING
plt.style.use('ggplot')
#display maximum column and rows
pd.options.display.max_columns = 999
pd.options.display.max_rows = 999
red = '#ef4b4b'
black = '#584b42'
green = '#a0cc78'
yellow_ = '#f6d365'
blue_ = '#49beb7'


#METHODS
def connectMongoDB(dbname):
    client = pymongo.MongoClient('localhost', 27017)
    db = client[dbname]
    return db

dbname = 'clothing_fit'
db = connectMongoDB(dbname)

# Sebelumnya data dalam bentuk JSON sudah diimport ke dalam MongoDB

- setting DB lokal 
        dbname = 'clothing_fit'
- nama collection -> rentCollection

In [2]:
rent = list(db.rentCollection.find())
df_rent = pd.DataFrame(rent)

# MENGECEK NULL
df_rent.head(1)

,_id,fit,user_id,bust size,item_id,weight,rating,rented for,review_text,body type,review_summary,category,height,size,age,review_date,sentiment_,height_cm,weight_kg,age_,review_date_,sentiment_summary,sentiment_text
0,5e2ab8ce9f84980caca61bf7,fit,420272,34d,2260466,137lbs,10.0,vacation,An adorable romper! Belt and zipper were a lit...,hourglass,So many compliments!,romper,"5' 8""",14,28,"April 20, 2016",positive,173.0,62.0,28.0,2016-04-20,positive,positive


# Convert Height menjadi Cm

Kolom 'Height' memiliki value string yang menyatakan tinggi badan seseorang.
5' 8" -> data ini disimpan dalam satuan kaki + inch

oleh karena itu untuk mengolah menjadi angka perlu diubah menjadi numerical data.

Di sini saya memakai satuan cm

In [3]:
df_rent = df_rent.dropna(subset=['height'])

In [4]:
# CONVERT (FEET INCH) INTO CM (common unit in Indonesia)

height_cm = []
for i in df_rent['height']:
    h_ft = int(i[0])
    h_inch = int(i[3])
    h_inch += h_ft * 12
    h_cm = round(h_inch * 2.54, 0)
    
    height_cm.append(h_cm)
df_rent['height_cm'] = height_cm
# df_rent.drop(['height'], axis=1)
df_rent.head()
df_rent.isnull().sum()

_id                     0
fit                     0
user_id                 0
bust size            1885
item_id                 0
weight               3140
rating                  3
rented for              0
review_text             0
body type            1501
review_summary          0
category                0
height                  0
size                    0
age                   103
review_date             0
sentiment_              0
height_cm               0
weight_kg            3140
age_                    0
review_date_            0
sentiment_summary       0
sentiment_text          0
dtype: int64

# UPDATE height_cm ke MONGODB

In [67]:
# UPDATE DATA DITAMBAH DENGAN DATA height transformed
dict_rent_transpose = df_rent.transpose().to_dict()
keys = dict_rent_transpose.keys() 
print("akan update sejumlah : ",len(dict_rent_transpose.keys()))


akan update sejumlah :  19951


In [68]:
# UPDATING data di mongodb

for i in keys:
    old = rent[i]
    new = {"$set": dict_rent_transpose[i]}
    db.rentCollection.update_one(old, new)

In [69]:
# MENGECEK HASIL UPDATE

rent = list(db.rentCollection.find())
df_rent = pd.DataFrame(rent)

# MENGECEK NULL
df_rent.isnull().sum()


_id                  0
fit                  0
user_id              0
bust size         1901
item_id              0
weight            3161
rating               6
rented for           2
review_text          0
body type         1510
review_summary       0
category             0
height              76
size                 0
age                103
review_date          0
sentiment_           0
height_cm           76
weight_kg         3216
dtype: int64

# convert weight menjadi KG

In [5]:
df_rent = df_rent.dropna(subset=['weight'])

In [6]:
# CONVERT POUNDS INTO KG (common unit in Indonesia)

df_rent['weight_kg'] = df_rent.weight.apply(lambda x : round(int(x[:-3])*0.45359237,0))
df_rent.head(3)

,_id,fit,user_id,bust size,item_id,weight,rating,rented for,review_text,body type,review_summary,category,height,size,age,review_date,sentiment_,height_cm,weight_kg,age_,review_date_,sentiment_summary,sentiment_text
0,5e2ab8ce9f84980caca61bf7,fit,420272,34d,2260466,137lbs,10.0,vacation,An adorable romper! Belt and zipper were a lit...,hourglass,So many compliments!,romper,"5' 8""",14,28,"April 20, 2016",positive,173.0,62.0,28.0,2016-04-20,positive,positive
1,5e2ab8ce9f84980caca61bf8,fit,273551,34b,153475,132lbs,10.0,other,I rented this dress for a photo shoot. The the...,straight & narrow,I felt so glamourous!!!,gown,"5' 6""",12,36,"June 18, 2013",positive,168.0,60.0,36.0,2013-06-18,positive,positive
3,5e2ab8ce9f84980caca61bfa,fit,909926,34c,126335,135lbs,8.0,formal affair,I rented this for my company's black tie award...,pear,Dress arrived on time and in perfect condition.,dress,"5' 5""",8,34,"February 12, 2014",positive,165.0,61.0,34.0,2014-02-12,positive,positive


# UPDATE weight_kg ke MONGODB

In [72]:
# UPDATE DATA DITAMBAH DENGAN DATA weight  transformed
dict_rent_transpose = df_rent.transpose().to_dict()
keys = dict_rent_transpose.keys() 
print("akan update sejumlah : ",len(dict_rent_transpose.keys()))


akan update sejumlah :  16866


In [73]:
# UPDATING data di mongodb
for i in keys:
    old = rent[i]
    new = {"$set": dict_rent_transpose[i]}
    db.rentCollection.update_one(old, new)

In [74]:
# MENGECEK HASIL UPDATE

rent = list(db.rentCollection.find())
df_rent = pd.DataFrame(rent)

# MENGECEK NULL
df_rent.isnull().sum()


_id                  0
fit                  0
user_id              0
bust size         1901
item_id              0
weight            3161
rating               6
rented for           2
review_text          0
body type         1510
review_summary       0
category             0
height              76
size                 0
age                103
review_date          0
sentiment_           0
height_cm           76
weight_kg         3161
dtype: int64


# Convert rating into float

In [88]:
df_rent['rating'] = df_rent['rating'].astype('float')
df_rent['rating']

dtype('float64')

# UPDATE rating float ke MONGODB

In [89]:
# UPDATE DATA DITAMBAH DENGAN DATA rating  transformed
dict_rent_transpose = df_rent.transpose().to_dict()
keys = dict_rent_transpose.keys() 
print("akan update sejumlah : ",len(dict_rent_transpose.keys()))


# UPDATING data di mongodb
for i in keys:
    old = rent[i]
    new = {"$set": dict_rent_transpose[i]}
    db.rentCollection.update_one(old, new)

akan update sejumlah :  20027


In [95]:
# MENGECEK HASIL UPDATE

rent = list(db.rentCollection.find())
df_rent = pd.DataFrame(rent)

# MENGECEK NULL
df_rent.isnull().sum()
df_rent.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20027 entries, 0 to 20026
Data columns (total 19 columns):
_id               20027 non-null object
fit               20027 non-null object
user_id           20027 non-null object
bust size         18126 non-null object
item_id           20027 non-null object
weight            16866 non-null object
rating            20021 non-null float64
rented for        20025 non-null object
review_text       20027 non-null object
body type         18517 non-null object
review_summary    20027 non-null object
category          20027 non-null object
height            19951 non-null object
size              20027 non-null int64
age               19924 non-null object
review_date       20027 non-null object
sentiment_        20027 non-null object
height_cm         19951 non-null float64
weight_kg         16866 non-null float64
dtypes: float64(3), int64(1), object(15)
memory usage: 2.9+ MB


# Transform Kolom age menjadi float

In [32]:
df_rent['age_'] = df_rent['age'].astype('float')
df_rent.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20027 entries, 0 to 20026
Data columns (total 21 columns):
_id               20027 non-null object
fit               20027 non-null object
user_id           20027 non-null object
bust size         18126 non-null object
item_id           20027 non-null object
weight            16866 non-null object
rating            20021 non-null float64
rented for        20025 non-null object
review_text       20027 non-null object
body type         18517 non-null object
review_summary    20027 non-null object
category          20027 non-null object
height            19951 non-null object
size              20027 non-null int64
age               19924 non-null object
review_date       20027 non-null object
sentiment_        20027 non-null object
height_cm         19951 non-null float64
weight_kg         16866 non-null float64
review_date_      20027 non-null datetime64[ns]
age_              19924 non-null float64
dtypes: datetime64[ns](1), float64(4), i

# Transform Kolom review_date menjadi datetime

In [9]:
df_rent['review_date'].dtype
df_rent['review_date'].head()


0        April 20, 2016
1         June 18, 2013
2     December 14, 2015
3     February 12, 2014
4    September 26, 2016
Name: review_date, dtype: object

In [24]:
import datetime

date_ = df_rent['review_date'][0]
date_ 
# date_time_str = 'Jun 28, 2018'
date_time_str = df_rent['review_date'][2]
date_time_obj = datetime.datetime.strptime(date_time_str, '%B %d, %Y')
date_time_obj

datetime.datetime(2015, 12, 14, 0, 0)

In [25]:
print('Date:', date_time_obj.date())
print('Time:', date_time_obj.time())
print('Date-time:', date_time_obj)

Date: 2015-12-14
Time: 00:00:00
Date-time: 2015-12-14 00:00:00


In [33]:
df_rent['review_date_'] = df_rent['review_date'].apply(lambda x : datetime.datetime.strptime(x, '%B %d, %Y'))
df_rent[['review_date', 'review_date_']]

df_rent.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20027 entries, 0 to 20026
Data columns (total 21 columns):
_id               20027 non-null object
fit               20027 non-null object
user_id           20027 non-null object
bust size         18126 non-null object
item_id           20027 non-null object
weight            16866 non-null object
rating            20021 non-null float64
rented for        20025 non-null object
review_text       20027 non-null object
body type         18517 non-null object
review_summary    20027 non-null object
category          20027 non-null object
height            19951 non-null object
size              20027 non-null int64
age               19924 non-null object
review_date       20027 non-null object
sentiment_        20027 non-null object
height_cm         19951 non-null float64
weight_kg         16866 non-null float64
review_date_      20027 non-null datetime64[ns]
age_              19924 non-null float64
dtypes: datetime64[ns](1), float64(4), i

# Update age_ dan review_date_ ke dalam MongoDB

In [34]:
# UPDATE DATA DITAMBAH DENGAN DATA weight  transformed
dict_rent_transpose = df_rent.transpose().to_dict()
keys = dict_rent_transpose.keys() 
print("akan update sejumlah : ",len(dict_rent_transpose.keys()))


akan update sejumlah :  20027


In [35]:
# UPDATING data di mongodb
for i in keys:
    old = rent[i]
    new = {"$set": dict_rent_transpose[i]}
    db.rentCollection.update_one(old, new)

In [ ]:
# MENGECEK HASIL UPDATE

rent = list(db.rentCollection.find())
df_rent = pd.DataFrame(rent)

# MENGECEK NULL
df_rent.isnull().sum()
df_rent.info()
